In [ ]:
import numpy as np
import matplotlib.pyplot as plt

gamma = 0.99
alpha = 0.2

In [ ]:
Q = np.load("Q.npy")
C = np.load("C.npy")

### Original implementation of Monte-Carlo method

In [ ]:
def getGreedyPolicy(Q):
    return np.argmax(Q,axis=-1)

def innerMCControl(episode, Q, C, t_policy, getBPolicy):
    b_policy = getBPolicy(Q)
    G = 0.0
    W = 1.0
    converged = True
    step = len(episode)-1
    for state, action, reward in reversed(episode):
        G = gamma*G + reward
        C[state][action] += W
        q = Q[state][action]
        Q[state][action] += W/C[state][action] * (G-Q[state][action])
        converged = converged and abs(q - Q[state][action]) < CONVERGENCE_ERROR
        t_policy[state] = np.argmax(Q[state])
        if t_policy[state] != action: break
        W *= 1.0/b_policy[state][action]
        step -= 1
    
    print("\repisode length:{:7}; steps used:{:3}".format(len(episode),len(episode)-step), 
          end='', flush=True)
    return converged

In [ ]:
def MCControl(episodes,C,getBPolicy,QShape):
    C = np.zeros(dtype=np.float, shape=episodes.QShape)
    t_policy = getGreedyPolicy(episodes.Q)
    for episode in episodes:
        innerMCControl(episode, episodes.Q, C, t_policy, lambda Q: episodes.policy)

#### Some geometry

In [ ]:
def getVectors(base, line):
    return np.array([line[0]-base, line[1]-base])
    
def det(v1,v2):
    return v1[0]*v2[1]-v1[1]*v2[0]

def is_intersection(l1, l2):
    return (det(*getVectors(l1[0],l2)) < 0) != (det(*getVectors(l1[1],l2)) < 0) \
           and (det(*getVectors(l2[0],l1)) < 0) != (det(*getVectors(l2[1],l1)) < 0)

### Implementation of RaceTrack environment

In [ ]:
MAX_VELOCITY = 4
MIN_VELOCITY = 0

rt_contour_1 = [
    [3,0],[3,3],[2,3],[2,10],[1,10],[1,18],[0,18],[0,28],[1,28],[1,29],[2,29],
    [2,31],[3,31],[3,32],[17,32],[17,26],[10,26],[10,25],[9,25],[9,0],[3,0]
]
start_line_1 = (3,0,9,1)
finish_line_1 = (16,26,17,32)
ACCELERATION =  [[1,-1],  [1,0],  [1,1],
                 [0,-1],  [0,0],  [0,1],
                [-1,-1], [-1,0], [-1,1]]
ACTIONS_NUM = len(ACCELERATION)

REWARD = -1
CONVERGENCE_ERROR = 0.02


In [ ]:
def rt_getStartPosition(start_line):
    return (0,np.random.randint(start_line[0], start_line[2]),0,0)

def is_finished(finish_line, position, next_position):
    return is_intersection(np.array([(finish_line[0],finish_line[1]),
                                     (finish_line[0],finish_line[3])]),
                           np.array([np.array(position)+0.5, 
                                     np.array(next_position)+0.5]))

def is_runout(contour, position, next_position):
    it = iter(contour)
    p1 = next(it)
    for p2 in it:
        if is_intersection(np.array([p1,p2]), 
                           np.array([np.array(position)+0.5, 
                                     np.array(next_position)+0.5])):
            return True
        p1 = p2
    return False

def rt_getTransition(track_contour, state, action, finish_line, getStartPosition=lambda: rt_getStartPosition(start_line_1)):
    accel = ACCELERATION[action]
    next_velocity = np.clip([state[2]+accel[0],state[3]+accel[1]], 
                            MIN_VELOCITY, MAX_VELOCITY)
    if (next_velocity == [0,0]).all():
        next_velocity = [state[2],state[3]]
    position = [state[0],state[1]]
    next_position = position + next_velocity

    if is_finished(finish_line, position, next_position):
        return (True,getStartPosition())

    if not is_runout(track_contour, position, next_position):
        next_state = tuple(next_position) + tuple(next_velocity)
    else:
        next_state = getStartPosition()
        
    return (False,next_state)
    

#### Test of RaceTrack environment

In [ ]:
is_intersection(np.array([(finish_line_1[0],finish_line_1[1]),
                                     (finish_line_1[2],finish_line_1[1])]),
                           np.array([np.array([4,26])+0.5, 
                                     np.array([18,30])+0.5]))

In [ ]:
[(finish_line_1[0],finish_line_1[1]),
                                     (finish_line_1[0],finish_line_1[3])]

In [ ]:
rt_getTransition(rt_contour_1, (12,30,3,1), 8, finish_line_1, 
                 getStartPosition=lambda: rt_getStartPosition(start_line_1))

In [ ]:
it = iter(contour)
p1 = next(it)
for p2 in it:
    print((p1,p2))
    if is_intersection(np.array([p1,p2]), 
                       np.array([np.array(position)+0.5, 
                                 np.array(next_position)+0.5])):

### Test line intersections

In [ ]:
l1 = np.array([(3.5,0.5),(2.5,5.5)])
l2 = np.array([(2,8),(1,8)])
l3 = np.array([(4,2.5),(4,4)])
l4 = np.array([(5,1.5),(5,3)])

plt.plot([0],[0],'bo', 
         l1[:,0], l1[:,1], 'r', 
         l2[:,0], l2[:,1], 'y', 
         l3[:,0], l3[:,1], 'm',
         l4[:,0], l4[:,1], 'g',)

# l1 and l2 intersect
print(det(*getVectors(l1[0],l2)), det(*getVectors(l1[1],l2))) 
print(det(*getVectors(l2[0],l1)), det(*getVectors(l2[1],l1)))
# l3 and l2 do not intersect
print(det(*getVectors(l3[0],l2)), det(*getVectors(l3[1],l2)))
print(det(*getVectors(l2[0],l3)), det(*getVectors(l2[1],l3)))
# l1 and l3 intersect
print(det(*getVectors(l1[0],l3)), det(*getVectors(l1[1],l3)))
print(det(*getVectors(l3[0],l1)), det(*getVectors(l3[1],l1)))

# l4 does not intersect with the rest
print("l4 intersections:")
print(det(*getVectors(l4[0],l3)), det(*getVectors(l4[1],l3)))
print(det(*getVectors(l4[0],l1)), det(*getVectors(l4[1],l1)))
print(det(*getVectors(l4[0],l2)), det(*getVectors(l4[1],l2)))
print(det(*getVectors(l1[0],l4)), det(*getVectors(l1[1],l4))) 
print(det(*getVectors(l2[0],l4)), det(*getVectors(l2[1],l4)))
print(det(*getVectors(l3[0],l4)), det(*getVectors(l3[1],l4)))

print((det(*getVectors(l1[0],l2)) < 0) != (det(*getVectors(l1[1],l2)) < 0) \
           and (det(*getVectors(l2[0],l1)) < 0) != (det(*getVectors(l2[1],l1)) < 0))

[is_intersection(l1,l2), is_intersection(l1,l3), is_intersection(l1,l4), 
 is_intersection(l2,l3), is_intersection(l2,l4), is_intersection(l3,l4),
 is_intersection(l2,l1), is_intersection(l3,l1), is_intersection(l4,l1),
 is_intersection(l3,l2), is_intersection(l4,l2), is_intersection(l4,l3)]

In [ ]:
det(*getVectors((2,8),l1))

### Original implementation of ε-greedy policy (for history)

In [ ]:
def getBPolicy(Q, epsilon):
    eps_policy = np.ones(dtype=np.float, shape=Q.shape) * epsilon/ACTIONS_NUM
    for state in np.ndindex(Q.shape[:-1]):
        action = np.argmax(Q[state])
        eps_policy[state][action] += 1.0-epsilon
    return eps_policy

def getBPolicyLog(Q, epsilon, step):
    e = min(1.0, epsilon/np.log(max(2,step/100)))
    eps_policy = np.ones(dtype=np.float, shape=Q.shape) * e/ACTIONS_NUM
    for state in np.ndindex(Q.shape[:-1]):
        action = np.argmax(Q[state])
        eps_policy[state][action] += 1.0-e
    return eps_policy


### Run Racetrack learning 

In [ ]:
import sys
sys.path.append('..')
import SeqGen


In [ ]:
sequence = SeqGen.SequenceGeneratorPlus(
                                SeqGen.EpsilonGreedyPolicy(Q, 0.05),
                                rt_getStartPosition,
                                lambda s,a: rt_getTransition(track, s, a) + (REWARD,),
                                10000
                               )

In [ ]:
def MC_run(N, Q, C):
    episodes = Episodes(N, Q, track, lambda q,s: getBPolicy(q,10000.0/(20000.0+s)))
                        #lambda q,s: rt_getBPolicy(q,0.5)) #lambda q,s: getBPolicyLog(q,0.7,s))
    t_policy = getGreedyPolicy(episodes.Q)
    conv_count = 0
    for episode in episodes:
        if innerMCControl(episode, episodes.Q, C, t_policy, lambda Q: episodes.policy):
            conv_count += 1
        else: 
            conv_count = 0
        if conv_count >= 500:
            print("\nConvergence reached.")
            break
            
    print("\nEpisodes generated: {}".format(episodes.samples_num - episodes.N))

In [ ]:
QShape = track.shape + (MAX_VELOCITY+1, MAX_VELOCITY+1, ACTIONS_NUM)
Q = (np.random.random(QShape)-0.5)*0.1 - 30.0
C = np.zeros(dtype=np.float, shape=Q.shape)
MC_run(40000, Q, C)

In [ ]:
MC_run(50000, Q, C)

In [ ]:
def buildEpisode(startPosition, policy):
    state = startPosition
    episode = []
    while True:
        action = policy[state]
        episode.append((state, action, reward))
        is_finished, next_state = rt_getTransition(track, state, action, lambda:startPosition)
        if is_finished or next_state==startPosition: break
        state = next_state
        
    return episode

def buildTrack(policy):
    return

In [ ]:
p = getGreedyPolicy(Q)
episode = buildEpisode((0,5,0,0),p)
last = episode[-1]
episode.append((tuple(np.array(last[0][:2])+np.array(last[0][2:])+ACCELERATION[last[1]]) +
               (0,0),0,len(episode)))

trace = np.array([list(s[:2]) for s,a,r in episode])
episode

### Plot the optimal paths

In [ ]:
class ImmutableGreedyPolicy:
    def __init__(self, Q):
        self.action = np.argmax(Q,axis=-1);
        
    def __call__(self, state):
        return self.action[state]
    

In [ ]:
epi = 3
def startPosition():
    global epi
    return (0,epi,0,0)


test_gen = SeqGen.SequenceGeneratorPlus(
    ImmutableGreedyPolicy(Q), 
    startPosition,
    lambda s,a: rt_getTransition(track, s, a, getStartPosition = lambda : (0,7,0,0)) + (REWARD,),
    episodes_max = 6,
    episode_maxlen = 20
    )

traces = []
episode = []
for state, is_terminal, next_state, action, reward in test_gen:
    episode.append(state[0:2])
    if is_terminal:
        traces.append(np.array(episode))
        episode = []
        epi += 1

In [ ]:
f = plt.figure(num=None, figsize=(8,10), dpi=80, facecolor='w', edgecolor='k')
plt.pcolor(track, figure=f)
cols = ['red','yellow','black','green','white','blue']
i = 0
for trace in traces:
    plt.plot(trace[:,1], trace[:,0], figure=f, linewidth=4.0, color=cols[i])
    i += 1

#### Some testing stuff

In [ ]:
QShape = track.shape + (MAX_VELOCITY+1, MAX_VELOCITY+1, ACTIONS_NUM)
Q = (np.random.random(QShape)-0.5)*0.1 - 30.0
episodes = Episodes(200, Q, track, lambda q,s: getBPolicyLog(q,0.7,s))
C = np.zeros(dtype=np.float, shape=episodes.Q.shape)
t_policy = getGreedyPolicy(episodes.Q)
conv_count = 0
for episode in episodes:
    if innerMCControl(episode, episodes.Q, C, t_policy, lambda Q: episodes.policy):
        conv_count += 1
    else: 
        conv_count = 0
    if conv_count > 100:
        print("Convergence reached.")
print("\nEpisodes generated: {}".format(episodes.samples_num - episodes.N))

In [ ]:
episodes = Episodes(2000, episodes.Q, track, lambda Q,s: rt_getBPolicy(Q,s,0.5))
for episode in episodes:
    innerMCControl(episode, episodes.Q, C, t_policy, lambda Q: episodes.policy)

In [ ]:
def rt_getTargetPolicyAction(Q, state):
    return np.argmax(Q[state])

In [ ]:
np.save("Q",Q)
np.save("C",C)

In [ ]:
epsilon = 0.5
getBPolicy = lambda Q: rt_getBPolicy(Q,epsilon)
MCControl()

In [ ]:
f,ns = rt_getTransition(track,state,action)
print(f,ns)
[(ACCELERATION[a],rt_getTransition(track,ns,a)) for a in range(9)]